### Shot-Detection

In [ ]:
# def detect_scenes(video_path, scene_list):
#     cap = cv2.VideoCapture(video_path)   
#     embeddings = []
#     metadatas = []
#     ids = []
#     with tqdm(total = len(scene_list), desc = "Processing frames") as pbar:
#         for i, scene in enumerate(scene_list):
#             start_time, end_time = scene[0].get_seconds(), scene[1].get_seconds()
#             mid_time = (start_time + end_time) / 2
#             timestamps = [start_time, mid_time, end_time]
#             labels = ["initial", "middle", "final"]
            
#             for t, label in zip(timestamps, labels):
#                 cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
#                 ret, frame = cap.read()
                
#                 if ret:
#                     img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                     image = Image.fromarray(img_rgb)
#                     inputs = processor(images = image, return_tensors = 'pt').to(device)
#                     blip_input = blip_processor(images = image, return_tensors = 'pt').to(device)
#                     with torch.no_grad():
#                         outputs = model.get_image_features(inputs.pixel_values)
#                         blip_outputs = blip_model.generate(**blip_input,
#                                                         # max_length = 500,
#                                                         # min_length = 150,
#                                                         # no_repeat_ngram_size=2,
#                                                         # num_beams = 5,
#                                                         )
                    
#                     caption = blip_processor.decode(blip_outputs[0], skip_special_tokens=True)
#                     image_embedding = outputs.pooler_output
#                     image_embedding = image_embedding / image_embedding.norm(dim = -1, keepdim= True)
#                     image_embedding = image_embedding.squeeze(0).cpu().numpy().tolist()
#                     timestamp_sec = t*1000
#                     frame_id = f"{video_path}:{timestamp_sec}"
                
#                     ids.append(frame_id)
#                     embeddings.append(image_embedding)

#                     metadatas.append({
#                         "frame_idx": f"frame_no_{i}_{label}",
#                         "caption": caption,
#                         "timestamp_ms": timestamp_sec,
#                         "source_path": video_path
#                     })
#             pbar.update(1)
                
#     return embeddings, metadatas, ids

In [ ]:
# def detect_scenes(video_path, scene_list):
#     cap = cv2.VideoCapture(video_path)   
#     embeddings = []
#     metadatas = []
#     ids = []
#     with tqdm(total = len(scene_list), desc = "Processing frames") as pbar:
#         for i, scene in enumerate(scene_list):
#             start_time, end_time = scene[0].get_seconds(), scene[1].get_seconds()
#             mid_time = (start_time + end_time) / 2
#             timestamps = mid_time
#             labels = "middle"
            
            
#             cap.set(cv2.CAP_PROP_POS_MSEC, timestamps * 1000)
#             ret, frame = cap.read()
            
#             if ret:
#                 img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                 image = Image.fromarray(img_rgb)
#                 inputs = processor(images = image, return_tensors = 'pt').to(device)
#                 blip_input = blip_processor(images = image, return_tensors = 'pt').to(device)
#                 with torch.no_grad():
#                     outputs = model.get_image_features(inputs.pixel_values)
#                     blip_outputs = blip_model.generate(**blip_input,
#                                                     # max_length = 60,
#                                                     # min_length = 20,
#                                                     # no_repeat_ngram_size=2,
#                                                     # num_beams = 5,
#                                                     )
                
#                 caption = blip_processor.decode(blip_outputs[0], skip_special_tokens=True)
#                 image_embedding = outputs.pooler_output
#                 image_embedding = image_embedding / image_embedding.norm(dim = -1, keepdim= True)
#                 image_embedding = image_embedding.squeeze(0).cpu().numpy().tolist()
#                 timestamp_sec = timestamps*1000
#                 frame_id = f"{video_path}:{timestamp_sec}"
            
#                 ids.append(frame_id)
#                 embeddings.append(image_embedding)

#                 metadatas.append({
#                     "frame_idx": f"frame_no_{i}_{labels}",
#                     "caption": caption,
#                     "timestamp_ms": timestamp_sec,
#                     "source_path": video_path
#                 })
#             pbar.update(1)
                
#     return embeddings, metadatas, ids

In [1]:
from scenedetect import open_video, SceneManager
import scenedetect
from scenedetect.detectors import ContentDetector
import os
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
import torch
from tqdm import tqdm
import cv2
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32",use_safetensors=True).to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

c:\Users\Owais\Downloads\video-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 398/398 [00:01<00:00, 273.08it/s, Materializing param=visual_projection.weight]                                
CLIPModel LOAD REPORT from: openai/clip-vit-base-patch32
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
vision_model.embeddings.position_ids | UNEXPECTED |  | 
text_model.embeddings.position_ids   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
The image processor of type `CLIPImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and m

In [ ]:
def scene_split(video_path):
    print("--- Detecting shot boundaries with PySceneDetect ---")
    video = open_video(video_path)
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector())

    try:
        scene_manager.detect_scenes(video, show_progress=True)
        scene_list = scene_manager.get_scene_list()
    except Exception as e:
        print("Scene detection failed:", e)
        scene_list = []
    return scene_list

In [ ]:
def generate_captions_in_batches(batch_of_frames, clip_model, clip_processor, blip_model, blip_processor, device="cuda"):
    clip_inputs = clip_processor(images=batch_of_frames, return_tensors="pt").to(device)
    blip_inputs = blip_processor(images = batch_of_frames, return_tensors = 'pt').to(device)
    with torch.no_grad():
        clip_outputs = clip_model.get_image_features(clip_inputs.pixel_values)
        clip_outputs = clip_outputs.pooler_output
        clip_embeddings = clip_outputs / clip_outputs.norm(p=2, dim=-1, keepdim=True)
        blip_output_ids = blip_model.generate(**blip_inputs,
                                        max_length = 80,
                                        min_length = 30,
                                        no_repeat_ngram_size=3,
                                        repetition_penalty=1.5,
                                        early_stopping=True,
                                        do_sample=False,
                                        num_beams = 5,
                                        )
        captions = blip_processor.batch_decode(blip_output_ids, skip_special_tokens=True)

   
            
    embeddings_list = clip_embeddings.cpu().numpy().tolist()
    del clip_inputs, blip_inputs, clip_outputs, blip_output_ids
    torch.cuda.empty_cache()        

    return captions, embeddings_list

In [5]:
def frame_listing(scene_list, video_path):
    cap = cv2.VideoCapture(video_path)   
    ids = []
    frame_PIL = []
    timestamps_list = []
    with tqdm(total = len(scene_list), desc="processing frames ") as pbar: 
        for i, scene in enumerate(scene_list):
            start_time, end_time = scene[0].get_seconds(), scene[1].get_seconds()
            mid_time = (start_time + end_time) / 2
            timestamps = [start_time, mid_time, end_time]
            labels = ["initial", "middle", "final"]
            
            for t, label in zip(timestamps, labels):
                cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
                ret, frame = cap.read()
                
                if ret:
                    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image = Image.fromarray(img_rgb)
                    frame_PIL.append(image)
                    timestamps_list.append(t)
                frame_id = f"{video_path}:{t}"
                ids.append(frame_id)
            pbar.update(1)
    return frame_PIL, timestamps_list, ids

In [6]:
def batched_captioning(frame_list: list, batch_size: int):
    caption_list = []
    embedding_list = []
    
    with tqdm(total = int(len(frame_list)/ batch_size), desc = "Batched image captioning") as pbar:
        for i in range(0, len(frame_list),batch_size):
            batch = frame_list[i:i+batch_size]
            caption , embedding = generate_captions_in_batches(batch, 
                                                            clip_model= model,
                                                            clip_processor= processor,
                                                            blip_model= blip_model,
                                                            blip_processor= blip_processor)
            caption_list.extend(caption)
            embedding_list.extend(embedding)
            pbar.update(1)
    return caption_list, embedding_list           


In [7]:
video_path = "../This Integral Breaks Math.mp4"

In [8]:
scene_list = scene_split("../This Integral Breaks Math.mp4")

--- Detecting shot boundaries with PySceneDetect ---


  Detected: 48 | Progress: 100%|██████████| 7793/7793 [01:06<00:00, 116.39frames/s]


In [9]:
frame_PIL, timestamps_list, ids = frame_listing(scene_list= scene_list, video_path= video_path)

processing frames : 100%|██████████| 49/49 [00:20<00:00,  2.43it/s]


In [10]:
caption_list, embeddings_list = batched_captioning(frame_list= frame_PIL, batch_size=16)

Batched image captioning:   0%|          | 0/9 [00:00<?, ?it/s]

Batched image captioning:  11%|█         | 1/9 [01:18<10:28, 78.52s/it]


KeyboardInterrupt: 

In [ ]:
len(caption_list), caption_list[:5]

(146,
 ['arafed image of a man making a sign with his fingers',
  'arafed image of a man in a gray shirt is making a face',
  'arafed image of a person writing a math problem on a blackboard',
  'arafed image of a person writing a math problem on a blackboard',
  'a close up of a black background with a white and yellow sign'])

In [49]:
# emb, met, ids = detect_scenes(video_path="../This Integral Breaks Math.mp4", scene_list= scene_list)

In [7]:
import chromadb
client = chromadb.PersistentClient(path = "../db_path")
collection = client.get_or_create_collection("frame_collection")

In [8]:
collection.add(
    ids = ids,
    embeddings= emb,
    metadatas= met,
)

In [13]:
w[0]

(00:00:00.000 [frame=0, fps=30.000], 00:00:01.233 [frame=37, fps=30.000])